In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6 = "
  data {
    int N;
    int T[N];
    int N_societies;
    int society[N];
    int P[N];
  }
  parameters {
    real alpha;
    vector[N_societies] a_society;
    real bp;
    real<lower=0> sigma_society;
  }
  model {
    vector[N] mu;
    target += normal_lpdf(alpha | 0, 10);
    target += normal_lpdf(bp | 0, 1);
    target += cauchy_lpdf(sigma_society | 0, 1);
    target += normal_lpdf(a_society | 0, sigma_society);
    for(i in 1:N) mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
    target += poisson_log_lpmf(T | mu);
  }
  generated quantities {
    vector[N] log_lik;
    {
    vector[N] mu;
    for(i in 1:N) {
      mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
      log_lik[i] = poisson_log_lpmf(T[i] | mu[i]);
    }
    }
  }
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6",  model=m12_6, output_format=:mcmcchain);

Input data for cmdstan

In [4]:
m12_6_data = Dict("N" => size(d, 1), "T" => d[:total_tools], "N_societies" => 10, "society" => d[:society], "P" => d[:population]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean          SD         Naive SE        MCSE         ESS    
         lp__ -39.676255075  3.232241663 0.05110622801 0.10871482605  883.954879
accept_stat__   0.944665726  0.075436651 0.00119275818 0.00140215516 1000.000000
   stepsize__   0.046693500  0.004053023 0.00006408392 0.00064892196   39.009752
  treedepth__   5.912750000  0.451872199 0.00714472681 0.01810577008  622.869756
 n_leapfrog__  74.400000000 34.788598156 0.55005603388 1.15701118050  904.063105
  divergent__   0.000000000  0.000000000 0.00000000000 0.00000000000         NaN
     energy__  46.231922925  4.151240687 0.06563687843 0.13636996704  926.655606
        alpha   1.133429178  0.743528572 0.01175621896 0.02115546042 1000.000000
  a_society.1  -0.213410365  0.248223007 0.00392475035 0.00663346725 1000.000000
  a_society.2   0.039249637  0.221347579 0.00349981252 0.00516250087 1000.

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*